# Setup

In [1]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-aeh0z10f
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-aeh0z10f
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-318s1cz5/pyfolio_1bd09ae66aca42ad855c665a5a87c185
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-318s1cz5/pyfolio_1bd09ae66aca42ad855c665a5a87c185
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-318s1cz5/elegantrl_ec74dd853d9243a4a658389d419668f6
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-318s1cz5/elegantrl_ec74dd853d9243a4a658389d419668f6


# StockPortEnv - environment for RL algorithms
It defines action, states, rewards, environment.

In [2]:
import gym
from gym.utils import seeding
from gym import spaces
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

# RL core function

In [3]:
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

# from StockPortfolioEnv import StockPortfolioEnv

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [26]:
def FinRL_PortfolioAllocation(
    start_date, end_date, train_end_date, ticker_list, lookback, model
):
    """
    A Portfolio Allocation algorithm using different Reinforcement Learning 
      models like A2C, PPO, DDPG, SAC, TD3.

    Parameters
    ----------
    start_date : int
        Starting date of training set. 
        Format: yyyy-mm-dd.
    end_date : str
        Ending date of training set; The day before starting date of trading set. 
        Format: yyyy-mm-dd.
    train_end_date : str
        Ending date of training set.
        Format: yyyy-mm-dd.
    ticker_list : list
        Ticker list of stocks. 
    lookback : int
        The lookback days (window range) of rolling windows of training. 
        It must be greater than date difference of sta  
    model: str
        The string of model name. 
        Available model: ["A2C","PPO","DDPG","SAC","TD3"]
        
    Returns
    -------
    df_daily_return : pd.DataFrame
        The daily returns during the trading period.
    df_actions : pd.DataFrame
        The portfolio allocation(weights) during the trading period.
    """
    df = YahooDownloader(
        start_date=start_date, end_date=end_date, ticker_list=ticker_list
    ).fetch_data()

    fe = FeatureEngineer(
        use_technical_indicator=True, use_turbulence=False, user_defined_feature=False
    )

    df = fe.preprocess_data(df)

    # add covariance matrix as states
    df = df.sort_values(["date", "tic"], ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    return_list = []

    lookback = lookback
    for i in range(lookback, len(df.index.unique())):
        data_lookback = df.loc[i - lookback : i, :]
        price_lookback = data_lookback.pivot_table(
            index="date", columns="tic", values="close"
        )
        return_lookback = price_lookback.pct_change().dropna()
        return_list.append(return_lookback)

        covs = return_lookback.cov().values
        cov_list.append(covs)
    df_cov = pd.DataFrame(
        {
            "date": df.date.unique()[lookback:],
            "cov_list": cov_list,
            "return_list": return_list,
        }
    )
    df = df.merge(df_cov, on="date")
    df = df.sort_values(["date", "tic"]).reset_index(drop=True)

    train = data_split(df, start_date, train_end_date)

    stock_dimension = len(train.tic.unique())
    state_space = stock_dimension
    # print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

    env_kwargs = {
        "hmax": 100,
        "initial_amount": 1000000,
        "transaction_cost_pct": 0.001,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4,
    }

    e_train_gym = StockPortfolioEnv(df=train, **env_kwargs)

    env_train, _ = e_train_gym.get_sb_env()
    # print(type(env_train))

    # initialize
    agent = DRLAgent(env=env_train)

    if model == "A2C":
        A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
        model_a2c = agent.get_model(model_name="a2c", model_kwargs=A2C_PARAMS)

        trained_model = agent.train_model(
            model=model_a2c, tb_log_name="a2c", total_timesteps=100000
        )
    elif model == "PPO":
        PPO_PARAMS = {
            "n_steps": 2048,
            "ent_coef": 0.005,
            "learning_rate": 0.0001,
            "batch_size": 128,
        }
        model_ppo = agent.get_model("ppo", model_kwargs=PPO_PARAMS)

        trained_model = agent.train_model(
            model=model_ppo, tb_log_name="ppo", total_timesteps=80
        )
    elif model == "DDPG":
        DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}
        model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS)

        trained_model = agent.train_model(
            model=model_ddpg, tb_log_name="ddpg", total_timesteps=2000
        )
    elif model == "SAC":
        SAC_PARAMS = {
            "batch_size": 128,
            "buffer_size": 100000,
            "learning_rate": 0.0003,
            "learning_starts": 100,
            "ent_coef": "auto_0.1",
        }
        model_sac = agent.get_model("sac", model_kwargs=SAC_PARAMS)

        trained_model = agent.train_model(
            model=model_sac, tb_log_name="sac", total_timesteps=5000
        )
    elif model == "TD3":
        TD3_PARAMS = {"batch_size": 100, "buffer_size": 1000000, "learning_rate": 0.001}
        model_td3 = agent.get_model("td3", model_kwargs=TD3_PARAMS)

        trained_model = agent.train_model(
            model=model_td3, tb_log_name="td3", total_timesteps=300
        )

    # The day before starting date of trading set 
    #   is the starting day of trading.
    trade = data_split(df, train_end_date, end_date)
    e_trade_gym = StockPortfolioEnv(df=trade, **env_kwargs)

    # DRLAgent.save(model+'.zip')

    df_daily_return, df_actions = DRLAgent.DRL_prediction(
        model=trained_model, environment=e_trade_gym
    )

    return df_daily_return, df_actions

# Test

In [27]:
# start_date,
# end_date,
# train_end_date,
# ticker_list,
# lookback,
# model
test_return, test_action = FinRL_PortfolioAllocation('2017-01-01', '2021-11-03', '2021-01-01', config.DOW_30_TICKER, 60, 'A2C')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
!pip install git+https://github.com/PawPol/PyPortOpt.git

  Cloning https://github.com/PawPol/PyPortOpt.git to /tmp/pip-req-build-10ctqxzk
  Running command git clone -q https://github.com/PawPol/PyPortOpt.git /tmp/pip-req-build-10ctqxzk


In [9]:
df = YahooDownloader(start_date = '2017-01-01',
                     end_date = '2021-11-03',
                     ticker_list = config.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [10]:
from PyPortOpt import Optimizers as o
# import pandas as pd
# import quantstats as qs

In [18]:
R, df_logret, w_all, rownames = o.rollingwindow_backtest(
            "minimumVariancePortfolio", df[['date', 'tic', 'close']], 60, 1
        )

In [19]:
return_s = pd.Series(R[-211:] / 100)
return_s.index = pd.DatetimeIndex(rownames[-211:])

In [28]:
return_drl = pd.Series(test_return["daily_return"])
return_drl.index = pd.DatetimeIndex(test_return["date"])

In [21]:
return_drl.shape, return_s.shape

((211,), (211,))

In [22]:
return_s.to_csv("return_minVar.csv")

In [29]:
return_drl.to_csv("return_drl.csv")

In [14]:
import quantstats as qs

In [30]:
qs.reports.html(return_drl, return_s, output="DRL_vs_MVP_3.html")